In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

/var/folders/kg/xp1kxddd3r365qjsy8z_p9lh0000gn/T/ipykernel_7082/1222050232.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
url = 'https://news.kbs.co.kr/news/pc/category/category.do?ref=pSiteMap#20240228&1'
html_doc = requests.get(url).text
html_doc

'<!DOCTYPE html>\n<html lang="ko" >\n<head>\n    \n\n    \n\n\n\n\n\n\n\n\n<!--  for global var  -->\n<script type="text/javascript">\n  var application = {"root_url":"https://news.kbs.co.kr","vod_cloud_front_url_block_international":"http://news-restrict.gscdn.kbs.co.kr","imagePath":"","resourcesPath":"/resources","root_domain":"news.kbs.co.kr","news_map_client_id":"udun63q80x","vod_api_url":"https://static.api.kbs.co.kr/play/1.2/sign","vodDefaultImage":"/resource/image/common/noimg_633_355.png","cdn_api_url_root":"https://static.api.kbs.co.kr","img_path_m_big":"https://img.kbs.co.kr/kbs/620/news.kbs.co.kr","cms_root_domain":"newscms.kbs.co.kr","vod_bucket":"newsvod.s3.kbs.co.kr","news_mobile_map_client_id":"vogmbm9h4v","jebo_root_url":"jebo.kbs.co.kr","defaultImage":"/resource/image/common/noimg_633_355.png","contextPath":"","rootDomain":"news.kbs.co.kr","img_path_m_sm":"https://img.kbs.co.kr/kbs/44/news.kbs.co.kr","img_path_m_thum":"https://img.kbs.co.kr/kbs/160/news.kbs.co.kr","res

In [4]:
soup = BeautifulSoup(html_doc, 'html.parser')
soup

<!DOCTYPE html>

<html lang="ko">
<head>
<!--  for global var  -->
<script type="text/javascript">
  var application = {"root_url":"https://news.kbs.co.kr","vod_cloud_front_url_block_international":"http://news-restrict.gscdn.kbs.co.kr","imagePath":"","resourcesPath":"/resources","root_domain":"news.kbs.co.kr","news_map_client_id":"udun63q80x","vod_api_url":"https://static.api.kbs.co.kr/play/1.2/sign","vodDefaultImage":"/resource/image/common/noimg_633_355.png","cdn_api_url_root":"https://static.api.kbs.co.kr","img_path_m_big":"https://img.kbs.co.kr/kbs/620/news.kbs.co.kr","cms_root_domain":"newscms.kbs.co.kr","vod_bucket":"newsvod.s3.kbs.co.kr","news_mobile_map_client_id":"vogmbm9h4v","jebo_root_url":"jebo.kbs.co.kr","defaultImage":"/resource/image/common/noimg_633_355.png","contextPath":"","rootDomain":"news.kbs.co.kr","img_path_m_sm":"https://img.kbs.co.kr/kbs/44/news.kbs.co.kr","img_path_m_thum":"https://img.kbs.co.kr/kbs/160/news.kbs.co.kr","resource_root_url":"//news.kbs.co.kr","

In [5]:
box_contents = soup.find_all('a', class_='box-content flex-style')
box_contents

[]

### 데이터가 담겨있지 않은 원인
 1. 크롤링하려는 태그들이 잘 설정되어있는지 확인
 2. 페이지 소스에 JavaScript가 사용되었는지 확인
 - 많은 웹페이지들이 : javascript 동적 데이터를 로드

### 이 경우, 해결책 = Selenium
 - Selenium 웹 페이지를 자동으로 제어, 테스트할 수 있는 도구. 직접적으로 웹 브라우저를 제어.
 - requests, beautifulsoup으로 크롤링 불가능할 때, 원하는 결과를 가져오지 못할 때 : selenium

### 정적 페이지 = Requests + Beautifulsoup
- 서버로부터 한 번 요청하면 변하지 않는 형태의 HTML로 작성된 페이지
- 웹 서버는 클라이언트 요청이 들어올 때마다 미리 만들어놓은 HTML 파일을 그대로 반환.
- 서버 측에서 미리 준비가 되어 있기 때문에, 클라이언트 측에서 별다른 처리 없이 바로 브라우저에 표시

### 동적 페이지 = Selenium
- 웹 서버로부터 HTML을 받아온 다음, 브라우저가 해석하고 실행하면서, Javascript 같은 스크립트 언어를 사용, 동적으로 페이지를 만듦
- 이렇게 생성된 HTML > 데이터를 불러오거나, 저희가 입력하거나 클릭, 스크롤을 할 때 > 화면이 변경되는 등의 이런 상호작용이 필요한 웹 페이지에서 사용.


In [8]:
# pip install selenium

In [10]:
# pip install webdriver_manager

In [11]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait

In [13]:
# Selenium으로 웹 드라이버를 실행
service = Service(executable_path=ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

# url 전달
url = 'https://news.kbs.co.kr/news/pc/category/category.do?ref=pSiteMap#20240228&1'
driver.get(url)

# 기다려달라는 값을 전달 : 최대 10초 기다리기
wait = WebDriverWait(driver, 10)

# 드라이버 접근 > 페이지 소스 가져오기
html = driver.page_source

# 드라이버 종료
driver.quit() 

In [14]:
soup = BeautifulSoup(html, 'html.parser')

In [15]:
box_contents = soup.find_all('a', class_='box-content flex-style')
box_contents

[<a class="box-content flex-style" href="/news/view.do?ncd=7901013">
 <div class="thumbnail">
 <img alt="" class="img" onerror="javascript:handleError(this);" src="https://news.kbs.co.kr/data/news/2024/02/28/20240228_bQ2qtP.jpg"/>
 </div>
 <div class="txt-wrapper">
 <p class="title">아동급식, 배달앱으로 비대면 주문…아동급식지원 플랫폼 운영 확대</p>
 <p class="news-text">급식지원을 받는 어린이들이 낙인 효과 등을 걱정하지 않고 배달앱을 통해 비대면으로 주문할 ...</p>
 <div class="field-writer">
 <span class="date">2024.02.28 (09:54)</span>
 </div>
 </div>
 </a>,
 <a class="box-content flex-style" href="/news/view.do?ncd=7901002">
 <div class="thumbnail">
 <img class="img" onerror="javascript:handleError(this);" src="https://news.kbs.co.kr/data/news/title_image/newsmp4/news930/2024/02/28/130_7901002.jpg"/>
 <span class="play-icon medium"></span>
 </div>
 <div class="txt-wrapper">
 <p class="title">프랑스, 눈 부족…‘더 높은’ 스키장으로</p>
 <p class="news-text"> 유럽에선 스키장들이 폐업을 고민해야 할 만큼 눈이 부족한 상황인데요. 이렇다 보니 스키 ...</p>
 <div class="field-writer">
 <span class="date">202

In [17]:
len(box_contents)

12

In [21]:
box_contents[0]

<a class="box-content flex-style" href="/news/view.do?ncd=7901013">
<div class="thumbnail">
<img alt="" class="img" onerror="javascript:handleError(this);" src="https://news.kbs.co.kr/data/news/2024/02/28/20240228_bQ2qtP.jpg"/>
</div>
<div class="txt-wrapper">
<p class="title">아동급식, 배달앱으로 비대면 주문…아동급식지원 플랫폼 운영 확대</p>
<p class="news-text">급식지원을 받는 어린이들이 낙인 효과 등을 걱정하지 않고 배달앱을 통해 비대면으로 주문할 ...</p>
<div class="field-writer">
<span class="date">2024.02.28 (09:54)</span>
</div>
</div>
</a>

In [22]:
url = "http://news.kbs.co.kr" + box_contents[0]['href']
url

'http://news.kbs.co.kr/news/view.do?ncd=7901013'

In [24]:
title = box_contents[0].find('p', class_='title').text
title

'아동급식, 배달앱으로 비대면 주문…아동급식지원 플랫폼 운영 확대'

In [25]:
body = box_contents[0].find('p', class_='news-text').text
body

'급식지원을 받는 어린이들이 낙인 효과 등을 걱정하지 않고 배달앱을 통해 비대면으로 주문할 ...'

In [26]:
date = box_contents[0].find('span', class_='date').text
date

'2024.02.28 (09:54)'

In [27]:
# 원하는 부분들만 찾아서, 리스트에 담고, 데이터프레임으로 만들기
# 빈 리스트 만들기
url_list=[]
title_list=[]
body_list=[]
date_list=[]

for box_content in box_contents:
    url = "http://news.kbs.co.kr" + box_content['href']
    title = box_content.find('p', class_='title').text
    body = box_content.find('p', class_='news-text').text
    date = box_content.find('span', class_='date').text

    url_list.append(url)
    title_list.append(title)
    body_list.append(body)
    date_list.append(date)

In [28]:
url_list

['http://news.kbs.co.kr/news/view.do?ncd=7901013',
 'http://news.kbs.co.kr/news/view.do?ncd=7901002',
 'http://news.kbs.co.kr/news/view.do?ncd=7901001',
 'http://news.kbs.co.kr/news/view.do?ncd=7901000',
 'http://news.kbs.co.kr/news/view.do?ncd=7900999',
 'http://news.kbs.co.kr/news/view.do?ncd=7900998',
 'http://news.kbs.co.kr/news/view.do?ncd=7900996',
 'http://news.kbs.co.kr/news/view.do?ncd=7900994',
 'http://news.kbs.co.kr/news/view.do?ncd=7900993',
 'http://news.kbs.co.kr/news/view.do?ncd=7900992',
 'http://news.kbs.co.kr/news/view.do?ncd=7900991',
 'http://news.kbs.co.kr/news/view.do?ncd=7900990']

In [29]:
title_list

['아동급식, 배달앱으로 비대면 주문…아동급식지원 플랫폼 운영 확대',
 '프랑스, 눈 부족…‘더 높은’ 스키장으로',
 'EDM 축제 ‘돈트 렛 대디 노우 코리아’ 4월 서울랜드서 열려',
 '오픈AI “NYT가 챗GPT 해킹…저작권 침해 사례 100건 만들어”',
 '동계올림픽 열렸던 곳인데…“진흙탕 된 스키장”',
 '“레이더에도 안 걸려”…드론 실전 배치 확산',
 '중국, 달 탐사 프로젝트 가속화',
 '코스피 2,628.83 코스닥 866.77',
 '“여자화장실서 왜 나와”…불법촬영 남성 멱살 잡고 버틴 여성',
 '나발니 시신 인도 도운 변호사 “구금됐다 풀려났다”',
 '정부, ‘토종 OTT 경쟁력 강화’ 지원 본격화',
 '바다로 던진 구호품에 목숨 건 쟁탈전…“가자 병원은 또 기능 상실”']

In [30]:
body_list

['급식지원을 받는 어린이들이 낙인 효과 등을 걱정하지 않고 배달앱을 통해 비대면으로 주문할 ...',
 ' 유럽에선 스키장들이 폐업을 고민해야 할 만큼 눈이 부족한 상황인데요. 이렇다 보니 스키 ...',
 '국내외 유명 DJ와 음악 프로듀서가 출연하는 EDM(일렉트로닉 댄스 뮤직) 축제 ‘돈트 렛...',
 '생성형 인공지능(AI) 챗GPT가 저작권을 침해했다며 미국 일간 뉴욕타임스(NYT)가 소송...',
 '보스니아 헤르체고비나의 수도 사라예보. 1984년 동계올림픽이  개최됐던 곳이지만 이번 겨...',
 '[앵커] 우크라이나와 러시아의 전쟁 등에서 소형 무인기, 드론의 쓰임새 커지면서 각국에서 ...',
 '  [앵커] 중국이 달 탐사 프로젝트에 속도를 내고 있습니다. [리포트] 중국은 자체 개발...',
 '',
 '[앵커] 한 20대 남성이 찜질방 여자 화장실에서 불법 촬영을 하다 경찰에 붙잡혔습니다. ...',
 '옥중 사망한 러시아 반정부 정치인 알렉세이 나발니의 변호사가 구금됐다가 풀려났다고 로이터통...',
 '정부의 토종 온라인 동영상 서비스(OTT) 지원이 본격화되고 있습니다. 문화체육관광부와 한...',
 ' [앵커] 장기화된 전쟁에 가자 지구 주민들의 상황은 갈수록 나빠지고 있습니다. 구호품 약...']

In [31]:
date_list

['2024.02.28 (09:54)',
 '2024.02.28 (09:48)',
 '2024.02.28 (09:47)',
 '2024.02.28 (09:47)',
 '2024.02.28 (09:46)',
 '2024.02.28 (09:46)',
 '2024.02.28 (09:45)',
 '2024.02.28 (09:43)',
 '2024.02.28 (09:43)',
 '2024.02.28 (09:40)',
 '2024.02.28 (09:40)',
 '2024.02.28 (09:39)']

In [32]:
# 데이터 저장
data = {'뉴스url':url_list, '제목':title_list, '내용':body_list, '날짜':date_list}
df = pd.DataFrame(data)
df

뉴스url  \
0   http://news.kbs.co.kr/news/view.do?ncd=7901013   
1   http://news.kbs.co.kr/news/view.do?ncd=7901002   
2   http://news.kbs.co.kr/news/view.do?ncd=7901001   
3   http://news.kbs.co.kr/news/view.do?ncd=7901000   
4   http://news.kbs.co.kr/news/view.do?ncd=7900999   
5   http://news.kbs.co.kr/news/view.do?ncd=7900998   
6   http://news.kbs.co.kr/news/view.do?ncd=7900996   
7   http://news.kbs.co.kr/news/view.do?ncd=7900994   
8   http://news.kbs.co.kr/news/view.do?ncd=7900993   
9   http://news.kbs.co.kr/news/view.do?ncd=7900992   
10  http://news.kbs.co.kr/news/view.do?ncd=7900991   
11  http://news.kbs.co.kr/news/view.do?ncd=7900990   

                                        제목  \
0      아동급식, 배달앱으로 비대면 주문…아동급식지원 플랫폼 운영 확대   
1                   프랑스, 눈 부족…‘더 높은’ 스키장으로   
2      EDM 축제 ‘돈트 렛 대디 노우 코리아’ 4월 서울랜드서 열려   
3   오픈AI “NYT가 챗GPT 해킹…저작권 침해 사례 100건 만들어”   
4                동계올림픽 열렸던 곳인데…“진흙탕 된 스키장”   
5                 “레이더에도 안 걸려”…드론 실전 배치 확산   
6                        중국, 달 탐사 프로젝트 가속화   
7                  코스피 2,628.83 코스닥 866.77   
8        “여자화장실서 왜 나와”…불법촬영 남성 멱살 잡고 버틴 여성   
9             나발니 시신 인도 도운 변호사 “구금됐다 풀려났다”   
10              정부, ‘토종 OTT 경쟁력 강화’ 지원 본격화   
11   바다로 던진 구호품에 목숨 건 쟁탈전…“가자 병원은 또 기능 상실”   

                                                   내용                  날짜  
0   급식지원을 받는 어린이들이 낙인 효과 등을 걱정하지 않고 배달앱을 통해 비대면으로 ...  2024.02.28 (09:54)  
1    유럽에선 스키장들이 폐업을 고민해야 할 만큼 눈이 부족한 상황인데요. 이렇다 보니...  2024.02.28 (09:48)  
2   국내외 유명 DJ와 음악 프로듀서가 출연하는 EDM(일렉트로닉 댄스 뮤직) 축제 ‘...  2024.02.28 (09:47)  
3   생성형 인공지능(AI) 챗GPT가 저작권을 침해했다며 미국 일간 뉴욕타임스(NYT)...  2024.02.28 (09:47)  
4   보스니아 헤르체고비나의 수도 사라예보. 1984년 동계올림픽이  개최됐던 곳이지만 ...  2024.02.28 (09:46)  
5   [앵커] 우크라이나와 러시아의 전쟁 등에서 소형 무인기, 드론의 쓰임새 커지면서 각...  2024.02.28 (09:46)  
6     [앵커] 중국이 달 탐사 프로젝트에 속도를 내고 있습니다. [리포트] 중국은 자...  2024.02.28 (09:45)  
7                                                      2024.02.28 (09:43)  
8   [앵커] 한 20대 남성이 찜질방 여자 화장실에서 불법 촬영을 하다 경찰에 붙잡혔습...  2024.02.28 (09:43)  
9   옥중 사망한 러시아 반정부 정치인 알렉세이 나발니의 변호사가 구금됐다가 풀려났다고 ...  2024.02.28 (09:40)  
10  정부의 토종 온라인 동영상 서비스(OTT) 지원이 본격화되고 있습니다. 문화체육관광...  2024.02.28 (09:40)  
11   [앵커] 장기화된 전쟁에 가자 지구 주민들의 상황은 갈수록 나빠지고 있습니다. 구...  2024.02.28 (09:39)

In [33]:
df.to_csv('news_kbs_box.csv', index=False)

In [35]:
# 페이지네이션 : 1페이지~13페이지 돌면서 크롤링하는 방법
# for문 f string 
for i in range(1,14):
    print(f"{i}번째 페이지")

1번째 페이지
2번째 페이지
3번째 페이지
4번째 페이지
5번째 페이지
6번째 페이지
7번째 페이지
8번째 페이지
9번째 페이지
10번째 페이지
11번째 페이지
12번째 페이지
13번째 페이지


In [36]:
# 빈 리스트 만들기
url_list=[]
title_list=[]
body_list=[]
date_list=[]

# 서비스 변수 설정
service = Service(executable_path=ChromeDriverManager().install())

# 1. 페이지를 돌면서 작업을 수행할 예정
for page_num in range(1,14):
    
    # Selenium으로 웹 드라이버를 실행
    driver = webdriver.Chrome(service=service)

    # url 전달
    url = f'https://news.kbs.co.kr/news/pc/category/category.do?ref=pSiteMap#20240228&{page_num}'
    driver.get(url)

    # 기다려달라는 값을 전달 : 최대 10초 기다리기
    wait = WebDriverWait(driver, 10)

    # 드라이버 접근 > 페이지 소스 가져오기
    html = driver.page_source

    # 드라이버 종료
    driver.quit() 

    # 2. Beautifulsoup 사용해서 파싱하고, 태그들을 찾아서 담아줌
    soup = BeautifulSoup(html, 'html.parser')
    box_contents = soup.find_all('a', class_ = 'box-content flex-style')

    # 3. 박스 데이터들을 하나씩 돌면서 리스트에 데이터를 담음
    for box_content in box_contents:
        url = "http://news.kbs.co.kr" + box_content['href']
        title = box_content.find('p', class_='title').text
        body = box_content.find('p', class_='news-text').text
        date = box_content.find('span', class_='date').text

        url_list.append(url)
        title_list.append(title)
        body_list.append(body)
        date_list.append(date)

# 데이터 저장
data = {'뉴스url':url_list, '제목':title_list, '내용':body_list, '날짜':date_list}
df = pd.DataFrame(data)

df.to_csv('news_kbs_pagenation.csv', index=False)